results 4.5.2

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
# from tensorflow import keras
import tensorflow as tf

# %load_ext tensorboard
# import tensorboard
import os

In [2]:
from datetime import datetime
tf.compat.v1.disable_eager_execution()


### Params

In [88]:
epochs = 20

### Model with grid search

In [89]:
best_val_loss = np.inf
best_model = -1 

In [ ]:
results = []
layer_1_neurons = [16,32,64]   ######## Check the number of neurons ########
layer_2_neurons = [16,32,64]  ######## Check the number of neurons ########
layer_3_neurons = [16,32,64]  ######## Check the number of neurons ########
reg_consts = [0]

### Defining evaluation metrics

In [3]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

In [4]:
def NSE(y_true, y_pred):

    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [92]:
#PDE Loss Function
def custom_loss(grads_inputs):
    du_dx, du_dt, dh_dx, fric_coeff, slope = grads_inputs[:,0], grads_inputs[:,1], grads_inputs[:,2], grads_inputs[:,3], grads_inputs[:,4]
    g = K.constant(9.8)
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def loss(y_true,y_pred):
        loss_saint_venant = du_dt + y_pred[:,0] * du_dx + g*dh_dx + g*slope + g*K.square(fric_coeff) * K.square(y_true[:,0])/(K.pow(y_true[:,1], 4/3) + K.epsilon())
        l = K.mean(K.square(loss_saint_venant))

        return 2*l+ K.sum(K.mean(K.square(y_pred - y_true), axis=0))
   
    # Return a function
    return loss

## Combined 2016-2017 Data

In [5]:
BASE_DIR_PATH = '/Users/neilb/Documents/dsci_thesis/Datasets'
DATASET_FILE = os.path.join(BASE_DIR_PATH, 'compiled_data_2016_2017.csv')

df_2016_2017 = pd.read_csv(DATASET_FILE)
df_2016_2017

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x
0,0,0,0,1,2,0,0,12.18,21.03,21.033407,14.842428,803.88,630.9,0.026165,0.023526,2016-01-01 00:00:00,0.0,14420
1,1,1,0,1,1,1,0,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 01:00:00,3600.0,14420
2,2,2,1,1,1,0,1,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 02:00:00,7200.0,14420
3,3,3,0,0,0,1,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 03:00:00,10800.0,14420
4,4,4,1,1,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 04:00:00,14400.0,14420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,16059,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 19:00:00,63140400.0,14420
17516,17516,16424,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 20:00:00,63144000.0,14420
17517,17517,16789,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 21:00:00,63147600.0,14420
17518,17518,17154,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 22:00:00,63151200.0,14420


In [6]:
df_2016_2017['friction_coeff'] = [0.033 for i in range(len(df_2016_2017))]
df_2016_2017['slope'] = [1/1500 for i in range(len(df_2016_2017))]

In [7]:
results

NameError: name 'results' is not defined

In [8]:
df_2016_2017

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x,friction_coeff,slope
0,0,0,0,1,2,0,0,12.18,21.03,21.033407,14.842428,803.88,630.9,0.026165,0.023526,2016-01-01 00:00:00,0.0,14420,0.033,0.000667
1,1,1,0,1,1,1,0,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 01:00:00,3600.0,14420,0.033,0.000667
2,2,2,1,1,1,0,1,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 02:00:00,7200.0,14420,0.033,0.000667
3,3,3,0,0,0,1,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 03:00:00,10800.0,14420,0.033,0.000667
4,4,4,1,1,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 04:00:00,14400.0,14420,0.033,0.000667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,16059,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 19:00:00,63140400.0,14420,0.033,0.000667
17516,17516,16424,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 20:00:00,63144000.0,14420,0.033,0.000667
17517,17517,16789,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 21:00:00,63147600.0,14420,0.033,0.000667
17518,17518,17154,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 22:00:00,63151200.0,14420,0.033,0.000667


In [ ]:
#split = int(0.50*len(df_2016_2017))
#split

In [ ]:
layer_1_neurons = [16,32,64]   ######## Check the number of neurons ########
layer_2_neurons = [16,32,64]  ######## Check the number of neurons ########
layer_3_neurons = [16,32,64]  ######## Check the number of neurons ########

In [9]:
train_2016_2017 = df_2016_2017[:int(0.50*len(df_2016_2017))]
val_2016_2017 = df_2016_2017[int(0.50*len(df_2016_2017)):int(0.75*len(df_2016_2017))]
test_2016_2017 = df_2016_2017[int(0.75*len(df_2016_2017)):]

In [10]:
train_2016_2017

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x,friction_coeff,slope
0,0,0,0,1,2,0,0,12.18,21.03,21.033407,14.842428,803.88,630.9,0.026165,0.023526,2016-01-01 00:00:00,0.0,14420,0.033,0.000667
1,1,1,0,1,1,1,0,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 01:00:00,3600.0,14420,0.033,0.000667
2,2,2,1,1,1,0,1,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 02:00:00,7200.0,14420,0.033,0.000667
3,3,3,0,0,0,1,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 03:00:00,10800.0,14420,0.033,0.000667
4,4,4,1,1,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 04:00:00,14400.0,14420,0.033,0.000667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8755,8755,0,0,0,0,0,12.53,21.24,31.158059,18.279046,826.98,637.2,0.037677,0.028687,2016-12-31 19:00:00,31604400.0,14420,0.033,0.000667
8756,8756,8756,0,0,0,0,0,12.53,21.27,31.158059,18.829763,826.98,638.1,0.037677,0.029509,2016-12-31 20:00:00,31608000.0,14420,0.033,0.000667
8757,8757,8757,0,0,0,0,0,12.53,21.29,31.158059,19.205911,826.98,638.7,0.037677,0.030070,2016-12-31 21:00:00,31611600.0,14420,0.033,0.000667
8758,8758,8758,0,0,0,0,0,12.53,21.31,31.158059,19.589424,826.98,639.3,0.037677,0.030642,2016-12-31 22:00:00,31615200.0,14420,0.033,0.000667


In [11]:
X_train_2016_2017 = np.array(train_2016_2017[['x','t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope', 'Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro']].values.tolist())
X_val_2016_2017 = np.array(val_2016_2017[['x','t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope', 'Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro']].values.tolist())
X_test_2016_2017 = np.array(test_2016_2017[['x','t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope', 'Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro']].values.tolist())
Y_train_2016_2017 = np.array(train_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']].values.tolist())
Y_val_2016_2017 = np.array(val_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']].values.tolist())
Y_test_2016_2017 = np.array(test_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']].values.tolist())


In [12]:
X_train_2016_2017

array([[1.44200000e+04, 0.00000000e+00, 2.10334073e+01, ...,
        2.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.44200000e+04, 3.60000000e+03, 2.12800722e+01, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.44200000e+04, 7.20000000e+03, 2.12800722e+01, ...,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [1.44200000e+04, 3.16116000e+07, 3.11580586e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.44200000e+04, 3.16152000e+07, 3.11580586e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.44200000e+04, 3.16188000e+07, 3.11580586e+01, ...,
        2.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [13]:
X_train_2016_2017.shape

(8760, 10)

In [12]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import sys

In [ ]:
def create_model(n1, n2, n3, reg):
    
    K.clear_session()
    
    model = Sequential()
    model.add(Dense(n1, activation='relu', kernel_regularizer=l2(reg), input_shape=(5,)))
    model.add(Dense(n2, activation='relu', kernel_regularizer=l2(reg)))
    model.add(Dense(n3, activation='relu', kernel_regularizer=l2(reg)))
    model.add(Dense(2))
    
    # Create input tensor
    input_tensor = Input(shape=(5,))
    
    # Get output tensor by calling the model on the input tensor
    output_tensor = model(input_tensor)
    
    # Calculate gradients within the same graph
    grads_u = K.gradients(output_tensor[:,0], input_tensor)[0]
    grads_h = K.gradients(output_tensor[:,1], input_tensor)[0]
    du_dx, du_dt, dh_dx = grads_u[:,0], grads_u[:,1], grads_h[:,0]
    calc_grads_inputs = K.stack((du_dx, du_dt, dh_dx, input_tensor[:,3], input_tensor[:,4]), axis=1)
    
    # Create new model that includes gradient computation
    full_model = Model(inputs=input_tensor, outputs=output_tensor)
    
    # Compile with custom loss that uses calc_grads_inputs
    full_model.compile(optimizer='adam', 
                      loss=[custom_loss(calc_grads_inputs)], 
                      metrics=['mape', 'mae', 'mse', NSE, r_square])
    
    return full_model

In [ ]:
for reg in reg_consts:
    for n1 in layer_1_neurons:
        for n2 in layer_2_neurons:
            for n3 in layer_3_neurons:
                print(n1,n2,n3,reg)
                K.clear_session()
                
                model = create_model(n1,n2,n3,reg)
                early_stopping_monitor = EarlyStopping(patience = 2, verbose=False)
                history = model.fit(X_train_2016_2017,Y_train_2016_2017, epochs=epochs, batch_size=32, validation_data=(X_val_2016_2017,Y_val_2016_2017), callbacks=[early_stopping_monitor])
                
                # Saving results
                val_loss = history.history['val_loss'][-1]
                val_mae = history.history['val_mae'][-1]
                val_mse = history.history['val_mse'][-1]
                val_mape = history.history['val_mape'][-1]
                val_nse = history.history['val_NSE'][-1]
                val_r_square = history.history['val_r_square'][-1]
                
                
                results = pd.DataFrame(columns=['n1','n2','n3', 'epochs', 'reg',
                               'val_r2', 'val_nse', 'val_mse', 'val_mae', 'val_mape'])
                
                # Create a new row as a DataFrame
                new_row = pd.DataFrame({
                    'n1': [n1],
                    'n2': [n2],
                    'n3': [n3], 
                    'epochs': [len(history.history['val_loss'])],
                    'reg': [reg],
                    'val_r2': [val_r_square], 
                    'val_nse': [val_nse], 
                    'val_mse': [val_mse], 
                    'val_loss': [val_loss],
                    'val_mae': [val_mae], 
                    'val_mape': [val_mape]
                })
                
                # Concatenate the new row with the existing results
                results = pd.concat([results, new_row], ignore_index=True)
                
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model = model
                    best_n1 = n1
                    best_n2 = n2
                    best_n3 = n3
                    best_reg = reg
                    best_history = history

## Original Mahesh Code

In [ ]:
for reg in reg_consts:
    for n1 in layer_1_neurons:
        for n2 in layer_2_neurons:
            for n3 in layer_3_neurons:
                print(n1,n2,n3,reg)
                K.clear_session()
                model = Sequential()
                model.add(Dense(n1, activation = 'relu', kernel_regularizer=l2(reg),input_shape = (5,)))
                model.add(Dense(n2, activation = 'relu', kernel_regularizer=l2(reg)))
                model.add(Dense(n3, activation = 'relu', kernel_regularizer=l2(reg)))

                model.add(Dense(2))
                grads_u = K.gradients(model.output[:,0], model.input)[0]
                grads_h = K.gradients(model.output[:,1], model.input)[0]


                du_dx, du_dt, dh_dx = grads_u[:,0],grads_u[:,1],grads_h[:,0]
                calc_grads_inputs = K.stack((du_dx, du_dt, dh_dx, model.input[:,3],model.input[:,4]), axis=1)
                # model.summary()
                #Compile the model
                model.compile(optimizer = 'adam', loss = [custom_loss(calc_grads_inputs)], metrics=['mape', 'mae', 'mse',NSE, r_square])
                #fit the model
                early_stopping_monitor = EarlyStopping(patience = 2, verbose=False)
                history = model.fit(X_train_2016_2017,Y_train_2016_2017, epochs=epochs, batch_size=128, validation_data=(X_val_2016_2017,Y_val_2016_2017), callbacks=[early_stopping_monitor])

                # Saving results
                val_loss = history.history['val_loss'][-1]
                val_mae = history.history['val_mae'][-1]
                val_mse = history.history['val_mse'][-1]
                val_mape = history.history['val_mape'][-1]
                val_nse = history.history['val_NSE'][-1]
                val_r_square = history.history['val_r_square'][-1]

                results.append({
                    'n1':n1,
                    'n2':n2,
                    'n3':n3, 
                    'epochs':len(history.history['val_loss']), 
                    'reg':reg,
                    'val_r2':val_r_square, 
                    'val_nse':val_nse, 
                    'val_mse':val_mse, 
                    'val_loss':val_loss,
                    'val_mae':val_mae, 
                    'val_mape':val_mape
                })
                
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model = model
                    best_n1 = n1
                    best_n2 = n2
                    best_n3 = n3
                    best_reg = reg
                    best_history = history
                    #model.save(save_model_path)
                    #results.to_csv(save_results_path)
                    
#results.to_csv(save_results_path)

In [ ]:
#PDE Loss Function
def custom_loss(grads_inputs):
    du_dx, du_dt, dh_dx, fric_coeff, slope = grads_inputs[:,0], grads_inputs[:,1], grads_inputs[:,2], grads_inputs[:,3], grads_inputs[:,4]
    g = K.constant(9.8)
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def loss(y_true,y_pred):
        loss_saint_venant = du_dt + y_pred[:,0] * du_dx + g*dh_dx + g*slope + g*K.square(fric_coeff) * K.square(y_true[:,0])/(K.pow(y_true[:,1], 4/3) + K.epsilon())
        l = K.mean(K.square(loss_saint_venant))

        return 2*l+ K.sum(K.mean(K.square(y_pred - y_true), axis=0))
   
    # Return a function
    return loss



model = Sequential()
#model.add(Dense(n1, activation = 'relu', input_shape = (1,)))
model.add(Dense(n1, activation = 'relu', kernel_regularizer=l2(reg),input_shape = (5,)))
model.add(Dense(n2, activation = 'relu', kernel_regularizer=l2(reg)))
model.add(Dense(n3, activation = 'relu', kernel_regularizer=l2(reg)))

model.add(Dense(2))
grads_u = K.gradients(model.output[:,0], model.input)[0]
grads_h = K.gradients(model.output[:,1], model.input)[0]


du_dx, du_dt, dh_dx = grads_u[:,0],grads_u[:,1],grads_h[:,0]
calc_grads_inputs = K.stack((du_dx, du_dt, dh_dx, model.input[:,3],model.input[:,4]), axis=1)
# model.summary()
#Compile the model
model.compile(optimizer = 'adam', loss = [custom_loss(calc_grads_inputs)], metrics=['mape', 'mae', 'mse',NSE, r_square])
#fit the model
early_stopping_monitor = EarlyStopping(patience = 2, verbose=False)
history = model.fit(X_train,Y_train, epochs=epochs, batch_size=128, validation_data=(X_val,Y_val), callbacks=[early_stopping_monitor])



In [ ]:
best_model.summary()

In [ ]:
best_model.evaluate(X_test_2016_2017, Y_test_2016_2017)

## Claude

In [98]:
import numpy as np
import tensorflow as tf

np.random.seed(30)
tf.random.set_seed(30)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
'''
def normalize_data(X_train, X_val, Y_train, Y_val):
    """Normalize input and output data"""
    X_mean = np.mean(X_train, axis=0)
    X_std = np.std(X_train, axis=0)
    Y_mean = np.mean(Y_train, axis=0)
    Y_std = np.std(Y_train, axis=0)
    
    X_train_norm = (X_train - X_mean) / (X_std + 1e-8)
    X_val_norm = (X_val - X_mean) / (X_std + 1e-8)
    Y_train_norm = (Y_train - Y_mean) / (Y_std + 1e-8)
    Y_val_norm = (Y_val - Y_mean) / (Y_std + 1e-8)
    
    return (X_train_norm, X_val_norm, Y_train_norm, Y_val_norm, 
            X_mean, X_std, Y_mean, Y_std)
'''
def custom_loss(grads_inputs, physics_weight=1.0):
    """Modified loss function with adjustable physics weight and improved numerical stability"""
    du_dx, du_dt, dh_dx, fric_coeff, slope = (grads_inputs[:,i] for i in range(5))
    g = K.constant(9.81)  # More precise gravity constant
    
    def loss(y_true, y_pred):
        # Data loss
        mse_loss = K.mean(K.square(y_pred - y_true), axis=0)
        data_loss = K.sum(mse_loss)
        
        # Physics loss with improved numerical stability
        u = y_pred[:,0]  # velocity
        h = y_pred[:,1]  # water level
        
        # Saint-Venant equation terms
        momentum_eq = (
            du_dt +                                         # ∂u/∂t
            u * du_dx +                                     # u∂u/∂x
            g * dh_dx +                                     # g∂h/∂x
            g * slope +                                     # gS₀
            g * K.square(fric_coeff) * K.square(u) /       # gn²u²/h^(4/3)
            (K.pow(K.maximum(h, K.epsilon()), 4/3) + K.epsilon())
        )
        
        physics_loss = K.mean(K.square(momentum_eq))
        
        # Combined loss with weighting
        return data_loss + physics_weight * physics_loss
    
    return loss

def create_pinn_model(input_dim, n1, n2, n3, reg_const):
    """Create PINN model with improved architecture"""
    model = Sequential([
        Dense(n1, activation='tanh', input_shape=(input_dim,)),
        Dense(n2, activation='tanh'),
        Dense(n3, activation='tanh'),
        Dense(2, activation='linear')
    ])

    return model

def train_pinn(X_train, Y_train, X_val, Y_val, n1=64, n2=32, n3=16, 
               reg_const=0, physics_weight=1.0, epochs=20, 
               batch_size=32, patience=2):
    """Train PINN with improved training process"""
    
    '''
    # Normalize data
    (X_train_norm, X_val_norm, Y_train_norm, Y_val_norm,
     X_mean, X_std, Y_mean, Y_std) = normalize_data(X_train, X_val, Y_train, Y_val)
    '''
    # Create and compile model
    model = create_pinn_model(X_train.shape[1], n1, n2, n3, reg_const)
    
    # Calculate gradients for physics loss
    grads_u = K.gradients(model.output[:,0], model.input)[0]
    grads_h = K.gradients(model.output[:,1], model.input)[0]
    du_dx, du_dt, dh_dx = grads_u[:,0], grads_u[:,1], grads_h[:,0]
    calc_grads_inputs = K.stack(
        (du_dx, du_dt, dh_dx, model.input[:,3], model.input[:,4]), 
        axis=1
    )
    
    # Compile with custom loss
    model.compile(
        optimizer='adam',
        loss=custom_loss(calc_grads_inputs, physics_weight),
        metrics=['mape', 'mae', 'mse', NSE, r_square]
    )
    
    # Train with early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, Y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, Y_val),
        callbacks=[early_stopping],
        verbose=1
    )
    
    loss, mape, mae, mse, nse, r2 = model.evaluate(X_val, Y_val)
    print(f'Validation Loss: {loss:.4f}')
    print(f'Validation MAPE: {mape:.4f}')
    print(f'Validation MAE: {mae:.4f}') 
    print(f'Validation MSE: {mse:.4f}')
    print(f'Validation NSE: {nse:.4f}')
    print(f'Validation R-squared: {r2:.4f}')

    return model, history

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

def hyperparameter_tuning(X_train, Y_train, X_val, Y_val, X_test, Y_test):
    # Define hyperparameter ranges
    layer_1_neurons = [64]
    layer_2_neurons = [64]
    layer_3_neurons = [64]
    reg_const = 0
    physics_weight = 1.0
    
    # Initialize results tracking
    results = pd.DataFrame(columns=[
        'n1', 'n2', 'n3', 
        'epochs', 'val_loss', 'val_mape', 'val_mae', 
        'val_mse', 'val_nse', 'val_r2'
    ])
    
    best_val_loss = float('inf')
    best_model = None
    best_params = None
    
    # Grid search
    for n1 in layer_1_neurons:
        for n2 in layer_2_neurons:
            for n3 in layer_3_neurons:
                print(f"\nTrying parameters: n1={n1}, n2={n2}, n3={n3}")
                # Clear previous model from memory
                K.clear_session()

                try:
                    # Train model with current parameters
                    model, history = train_pinn(
                        X_train, Y_train,
                        X_val, Y_val,
                        n1=n1, n2=n2, n3=n3,
                        reg_const=reg_const,
                        physics_weight=physics_weight,
                        epochs=20,
                        batch_size=32,
                        patience=2
                    )

                    # Evaluate model
                    val_metrics = model.evaluate(X_val, Y_val, verbose=0)
                    val_loss = val_metrics[0]

                    # Store results
                    new_row = pd.DataFrame({
                        'n1': [n1],
                        'n2': [n2],
                        'n3': [n3],
                        'epochs': [len(history.history['loss'])],
                        'val_loss': [val_metrics[0]],
                        'val_mape': [val_metrics[1]],
                        'val_mae': [val_metrics[2]],
                        'val_mse': [val_metrics[3]],
                        'val_nse': [val_metrics[4]],
                        'val_r2': [val_metrics[5]]
                    })

                    results = pd.concat([results, new_row], ignore_index=True)

                    # Update best model if current one is better
                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        best_model = model
                        best_params = {
                            'n1': n1,
                            'n2': n2,
                            'n3': n3
                        }

                    # Save intermediate results
                    results.to_csv('hyperparameter_search_results.csv', index=False)

                except Exception as e:
                    print(f"Error with parameters: {e}")
                    continue
    
    # Print best parameters and their performance
    print("\nBest parameters found:")
    for param, value in best_params.items():
        print(f"{param}: {value}")
    
    return best_model, results, best_params


In [ ]:

best_model, results, best_params = hyperparameter_tuning(
        X_train_2016_2017, Y_train_2016_2017, X_val_2016_2017, Y_val_2016_2017, X_test_2016_2017, Y_test_2016_2017
    )
    


In [ ]:
best_model.summary()

In [ ]:
results

In [ ]:
model, history = train_pinn(X_train_2016_2017, Y_train_2016_2017, X_val_2016_2017, Y_val_2016_2017, n1=64, n2=64, n3=64, 
               reg_const=0, physics_weight=1.0, epochs=20, 
               batch_size=32, patience=2)

In [ ]:
model.evaluate(X_test_2016_2017, Y_test_2016_2017)

## Try using LSTM for the PINNs

In [13]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

def custom_loss(grads_inputs, physics_weight=1.0):
    """Modified loss function with adjustable physics weight and improved numerical stability"""
    du_dx, du_dt, dh_dx, fric_coeff, slope = (grads_inputs[:,i] for i in range(5))
    g = K.constant(9.81)  # More precise gravity constant
    
    def loss(y_true, y_pred):
        # Data loss
        mse_loss = K.mean(K.square(y_pred - y_true), axis=0)
        data_loss = K.sum(mse_loss)
        
        # Physics loss with improved numerical stability
        u = y_pred[:,0]  # velocity
        h = y_pred[:,1]  # water level
        
        # Saint-Venant equation terms
        momentum_eq = (
            du_dt +                                         # ∂u/∂t
            u * du_dx +                                     # u∂u/∂x
            g * dh_dx +                                     # g∂h/∂x
            g * slope +                                     # gS₀
            g * K.square(fric_coeff) * K.square(y_true[:,0]) /       # gn²u²/h^(4/3)
            (K.pow(y_true[:,1], 4/3) + K.epsilon())
        )
        
        physics_loss = K.mean(K.square(momentum_eq))
        
        # Combined loss with weighting
        return data_loss + physics_weight * physics_loss
    
    return loss

def create_pinn_model(X_train, X_val, n1, n2, n3, reg_const):
    """Create PINN model with improved architecture"""
        
        
    lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((1, 10), input_shape=(10,)),  # Reshape to 3D
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Flatten(),  # Add Flatten layer to handle dimension mismatch
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
    ])

    return lstm_model

def train_pinn(X_train, Y_train, X_val, Y_val, n1=64, n2=64, n3=64, 
               reg_const=0, physics_weight=1.0, epochs=20, 
               batch_size=32, patience=2):
    """Train PINN with improved training process"""
    
    '''
    # Normalize data
    (X_train_norm, X_val_norm, Y_train_norm, Y_val_norm,
     X_mean, X_std, Y_mean, Y_std) = normalize_data(X_train, X_val, Y_train, Y_val)
    '''
    # Create and compile model
    model = create_pinn_model(X_train, X_val,n1, n2, n3, reg_const)
    
    # Calculate gradients for physics loss
    grads_u = K.gradients(model.output[:,0], model.input)[0]
    grads_h = K.gradients(model.output[:,1], model.input)[0]
    du_dx, du_dt, dh_dx = grads_u[:,0], grads_u[:,1], grads_h[:,0]
    calc_grads_inputs = K.stack(
        (du_dx, du_dt, dh_dx, model.input[:,3], model.input[:,4]), 
        axis=1
    )
    
    # Compile with custom loss
    model.compile(
        optimizer='adam',
        loss=custom_loss(calc_grads_inputs, physics_weight),
        metrics=['mape', 'mae', 'mse', NSE, r_square]
    )
    
    # Train with early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, Y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, Y_val),
        callbacks=[early_stopping],
        verbose=1
    )
    
    loss, mape, mae, mse, nse, r2 = model.evaluate(X_val, Y_val)
    print(f'Validation Loss: {loss:.4f}')
    print(f'Validation MAPE: {mape:.4f}')
    print(f'Validation MAE: {mae:.4f}') 
    print(f'Validation MSE: {mse:.4f}')
    print(f'Validation NSE: {nse:.4f}')
    print(f'Validation R-squared: {r2:.4f}')

    return model, history

In [14]:
model, history = train_pinn(
                        X_train_2016_2017, Y_train_2016_2017, X_val_2016_2017, Y_val_2016_2017,
                        epochs=20,
                        batch_size=32,
                        patience=2
                    )

Train on 8760 samples, validate on 4380 samples
Epoch 1/20
8760/8760 [==============================] - ETA: 0s - loss: 17.5002 - mape: 97.7870 - mae: 1.1493 - mse: 8.7501 - NSE: 0.7718 - r_square: 2.0338e-04

C:\Users\neilb\anaconda3\lib\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


8760/8760 [==============================] - 7s 854us/sample - loss: 17.5002 - mape: 97.7870 - mae: 1.1493 - mse: 8.7501 - NSE: 0.7718 - r_square: 2.0338e-04 - val_loss: 0.2530 - val_mape: 42.8248 - val_mae: 0.2442 - val_mse: 0.1265 - val_NSE: 0.9964 - val_r_square: 4.7346e-14
Epoch 2/20
8760/8760 [==============================] - 3s 387us/sample - loss: 0.4682 - mape: 46.4186 - mae: 0.2686 - mse: 0.2341 - NSE: 0.9939 - r_square: 3.3188e-04 - val_loss: 0.3055 - val_mape: 88.6811 - val_mae: 0.2725 - val_mse: 0.1527 - val_NSE: 0.9957 - val_r_square: 4.5860e-13
Epoch 3/20
8760/8760 [==============================] - 4s 436us/sample - loss: 0.4601 - mape: 42.1328 - mae: 0.2667 - mse: 0.2300 - NSE: 0.9940 - r_square: 1.4152e-04 - val_loss: 0.3513 - val_mape: 109.7405 - val_mae: 0.2921 - val_mse: 0.1757 - val_NSE: 0.9951 - val_r_square: 3.2168e-13
Validation Loss: 0.2530
Validation MAPE: 42.8248
Validation MAE: 0.2442
Validation MSE: 0.1265
Validation NSE: 0.9964
Validation R-squared: 0.000

In [15]:
model.evaluate(X_test_2016_2017, Y_test_2016_2017)

[0.3786140834164198,
 16.001654,
 0.20283087,
 0.18928564,
 0.9957709,
 1.6543009e-13]